# Multi-node multi-GPU example on Azure using dask-cloudprovider

[Dask Cloud Provider](https://cloudprovider.dask.org/en/latest/) is a native cloud intergration for dask. It helps manage Dask clusters on different cloud platforms. In this notebook, we will look at how we can use the package to set-up a Azure cluster and run a multi-node, multi-GPU example with [RAPIDS](https://rapids.ai/). RAPIDS provides a suite of libraries to accelerate data science pipelines on the GPU entirely. This can be scaled to multiple nodes using Dask as we will see through this notebook. 

For the purposes of this demo, we will use the a part of the NYC Taxi Dataset (only the files of 2014 calendar year will be used here). The goal is to predict the fare amount for a given trip given the times and coordinates of the taxi trip.

Before running the notebook, run the following commands in the terminal to setup Azure CLI
```
pip install azure-cli
az login
```
And follow the instructions on the prompt to finish setting up the account.

The list of packages needed for this notebook is listed in the cell below - uncomment and run the cell to set it up.

In [ ]:
# !pip install "dask-cloudprovider[azure]"
# !pip install "dask-cloudprovider[azure]" --upgrade
# !pip install --upgrade azure-mgmt-network azure-mgmt-compute
# !pip install gcsfs
# !pip install dask_xgboost
# !pip install azureml

In [ ]:
from dask.distributed import Client, wait
from dask_cuda import LocalCUDACluster
import dask_cudf
import numpy as np

# Azure cluster set up

Let us now setup the [Azure cluster](https://cloudprovider.dask.org/en/latest/azure.html) using `AzureVMCluster` from Dask Cloud Provider. To do this, you;ll first need to set up a Resource Group, a Virtual Network and a Security Group on Azure. [Learn more about how you can set this up](https://cloudprovider.dask.org/en/latest/azure.html#resource-groups). Note that you can also set it up using the Azure portal directly.

Once you have set it up, you can now plug in the names of the entities you have created in the cell below. Finally note that we use the RAPIDS docker image to build the VM and use the `dask_cuda.CUDAWorker` to run within the VM.

In [ ]:
location = "SOUTH CENTRAL US"
resource_group = "RAPIDS-HPO-test"
vnet = "dask-vnet"
security_group = "test-security-group"

vm_size = "Standard_NC12s_v3"
docker_image = "rapidsai/rapidsai-core:cuda10.2-runtime-ubuntu18.04-py3.8"
worker_class = "dask_cuda.CUDAWorker"
 
n_workers = 2
env_vars = {"EXTRA_PIP_PACKAGES": "gcsfs"}

In [ ]:
from dask_cloudprovider.azure import AzureVMCluster

cluster = AzureVMCluster(
    location=location,
    resource_group=resource_group,
    vnet=vnet,
    security_group=security_group,
    vm_size=vm_size,
    docker_image=docker_image,
    worker_class=worker_class,
    env_vars=env_vars,
)

Let's look at the data locally to see what we're dealing with. We will make use of the data from 2014 for the purposes of the demo. We see that there are columns for pickup and dropoff times, distance, along with latitude, longitude, etc. These are the information we'll use to estimate the trip fare amount.

In [ ]:
base_path = 'gcs://anaconda-public-data/nyc-taxi/csv/'
tmp_df = dask_cudf.read_csv(base_path+'2014/yellow_tripdata_2014*.csv', n_rows=1000)

tmp_df.head().to_pandas()

In [ ]:
# Let's delete the dataframe and free up some memory
del tmp_df

# Data Cleanup

The data needs to be cleaned up before it can be used in a meaningful way. We first perform a renaming of some columns to a cleaner name (for instance, some of the years have `tpep_ropoff_datetime` instead of `dropfoff_datetime`). We also define the datatypes each of the columns need to be read as.

In [ ]:
# list of column names that need to be re-mapped
remap = {}
remap['tpep_pickup_datetime'] = 'pickup_datetime'
remap['tpep_dropoff_datetime'] = 'dropoff_datetime'
remap['ratecodeid'] = 'rate_code'

#create a list of columns & dtypes the df must have
must_haves = {
 'pickup_datetime': 'datetime64[ms]',
 'dropoff_datetime': 'datetime64[ms]',
 'passenger_count': 'int32',
 'trip_distance': 'float32',
 'pickup_longitude': 'float32',
 'pickup_latitude': 'float32',
 'rate_code': 'int32',
 'dropoff_longitude': 'float32',
 'dropoff_latitude': 'float32',
 'fare_amount': 'float32'
}

In [ ]:
def clean(df_part, remap, must_haves):
    """
    This function performs the various clean up tasks for the data
    and returns the cleaned dataframe.
    """
    tmp = {col:col.strip().lower() for col in list(df_part.columns)}
    df_part = df_part.rename(columns=tmp)
    # rename using the supplied mapping
    df_part = df_part.rename(columns=remap)
    # iterate through columns in this df partition
    for col in df_part.columns:
        # drop anything not in our expected list
        if col not in must_haves:
            df_part = df_part.drop(col, axis=1)
            continue

        # fixes datetime error found by Ty Mckercher and fixed by Paul Mahler
        if df_part[col].dtype == 'object' and col in ['pickup_datetime', 'dropoff_datetime']:
            df_part[col] = df_part[col].astype('datetime64[ms]')
            continue

        # if column was read as a string, recast as float
        if df_part[col].dtype == 'object':
            df_part[col] = df_part[col].str.fillna('-1')
            df_part[col] = df_part[col].astype('float32')
        else:
            # downcast from 64bit to 32bit types
            # Tesla T4 are faster on 32bit ops
            if 'int' in str(df_part[col].dtype):
                df_part[col] = df_part[col].astype('int32')
            if 'float' in str(df_part[col].dtype):
                df_part[col] = df_part[col].astype('float32')
            df_part[col] = df_part[col].fillna(-1)
    return df_part

# Add Interesting Features

We'll add new features by making use of "uder defined functions" on the dataframe. We'll make use of [apply_rows](https://docs.rapids.ai/api/cudf/stable/api.html#cudf.core.dataframe.DataFrame.apply_rows), which is similar to Pandas' apply funciton. `apply_rows` operation is [JIT compiled by numba](https://numba.pydata.org/numba-doc/dev/cuda/kernels.html) into GPU kernels. 

The kernels we define are - 
1. Haversine distance: This is used for calculating the total trip distance.

2. Day of the week: This can be useful information for determining the fare cost.

`add_features` function combined the two to produce a new dataframe that has the added features.

In [ ]:
import math
from math import cos, sin, asin, sqrt, pi

def haversine_distance_kernel(pickup_latitude, pickup_longitude, dropoff_latitude, dropoff_longitude, h_distance):
    for i, (x_1, y_1, x_2, y_2) in enumerate(zip(pickup_latitude, pickup_longitude, dropoff_latitude, dropoff_longitude)):
        x_1 = pi/180 * x_1
        y_1 = pi/180 * y_1
        x_2 = pi/180 * x_2
        y_2 = pi/180 * y_2
        
        dlon = y_2 - y_1
        dlat = x_2 - x_1
        a = sin(dlat/2)**2 + cos(x_1) * cos(x_2) * sin(dlon/2)**2
        
        c = 2 * asin(sqrt(a)) 
        r = 6371 # Radius of earth in kilometers
        
        h_distance[i] = c * r

def day_of_the_week_kernel(day, month, year, day_of_week):
    for i, (d_1, m_1, y_1) in enumerate(zip(day, month, year)):
        if month[i] <3:
            shift = month[i]
        else:
            shift = 0
        Y = year[i] - (month[i] < 3)
        y = Y - 2000
        c = 20
        d = day[i]
        m = month[i] + shift + 1
        day_of_week[i] = (d + math.floor(m*2.6) + y + (y//4) + (c//4) -2*c)%7
        
def add_features(df):
    df['hour'] = df['pickup_datetime'].dt.hour
    df['year'] = df['pickup_datetime'].dt.year
    df['month'] = df['pickup_datetime'].dt.month
    df['day'] = df['pickup_datetime'].dt.day
    df['diff'] = df['dropoff_datetime'].astype('int32') - df['pickup_datetime'].astype('int32')
    
    df['pickup_latitude_r'] = df['pickup_latitude']//.01*.01
    df['pickup_longitude_r'] = df['pickup_longitude']//.01*.01
    df['dropoff_latitude_r'] = df['dropoff_latitude']//.01*.01
    df['dropoff_longitude_r'] = df['dropoff_longitude']//.01*.01
    
    df = df.drop('pickup_datetime', axis=1)
    df = df.drop('dropoff_datetime', axis =1)
    
    
    df = df.apply_rows(haversine_distance_kernel,
                   incols=['pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude'],
                   outcols=dict(h_distance=np.float32),
                   kwargs=dict())
    
    
    df = df.apply_rows(day_of_the_week_kernel,
                      incols=['day', 'month', 'year'],
                      outcols=dict(day_of_week=np.float32),
                      kwargs=dict())
    
    
    df['is_weekend'] = (df['day_of_week']<2)
    return df


# Train RF model

We are now ready to fit a Random Forest on the data to predict the fare for the trip.

In [ ]:
cu_rf_params = {
    'n_estimators': 100,
    'max_depth': 16,
}

The cell below creates a client with the cluster we defined earlier in the notebook. Note that we have `cluster.scale`. This is the step where the workers are allocated.

Once workers become available, we can now run the rest of our workflow - reading and cleaning the data, splitting into training and validation sets, fitting a RF model and predicting on the validation set. We print out the MSE metric for this problem. Note that for better performance we should perform HPO ideally. 


Refer to the notebooks in the repository for how to perform automated HPO [using RayTune](https://github.com/rapidsai/cloud-ml-examples/blob/main/ray/notebooks/Ray_RAPIDS_HPO.ipynb) and [using Optuna](https://github.com/rapidsai/cloud-ml-examples/blob/main/optuna/notebooks/optuna_rapids.ipynb).

In [ ]:
with Client(cluster) as client:
    
    print("Start Workflow")
    cluster.scale(2)
    client.wait_for_workers(4)
    print("Step 0 - Got Workers")
    print(client)
    
    base_path = 'gcs://anaconda-public-data/nyc-taxi/csv/'
    df_2014 = dask_cudf.read_csv(base_path+'2014/yellow_tripdata_2014*.csv', n_rows=20000)
    print("Step 1 - Finished reading file")

    df_2014 = clean(df_2014, remap, must_haves)
    
    # Query the dataframe to clean up the outliers 

    query_frags = [
        'fare_amount > 0 and fare_amount < 500',
        'passenger_count > 0 and passenger_count < 6',
        'pickup_longitude > -75 and pickup_longitude < -73',
        'dropoff_longitude > -75 and dropoff_longitude < -73',
        'pickup_latitude > 40 and pickup_latitude < 42',
        'dropoff_latitude > 40 and dropoff_latitude < 42'
    ]

    df_2014 = df_2014.query(' and '.join(query_frags))
    print("Step 2 - Cleaned data and removed outliers")

    taxi_df = df_2014.map_partitions(add_features)
    print("Step 3 - Added features")

    taxi_df = taxi_df.dropna()
    taxi_df = taxi_df.astype("float32")
    from dask_ml.model_selection import train_test_split
    from cuml.dask.common import utils as dask_utils
    X, y = taxi_df.drop(["fare_amount"], axis=1), taxi_df["fare_amount"].astype('float32')
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True)
    print("Step 4 - Split data")

    workers = client.has_what().keys()
    X_train, X_test, y_train, y_test = dask_utils.persist_across_workers(client,
                                                           [X_train, X_test, y_train, y_test],
                                                           workers=workers)

    from cuml.dask.ensemble import RandomForestRegressor
    cu_dask_rf = RandomForestRegressor(ignore_empty_partitions=True)
    cu_dask_rf = cu_dask_rf.fit(X_train, y_train)
    wait(cu_dask_rf.rfs)
    print("Step 5 - Fitted RF")

    y_pred = cu_dask_rf.predict(X_test)
    print("Step 6 - Predicted on test set")

    _y_pred, _y_test = y_pred.compute().to_array(), y_test.compute().to_array()

    print("Step 7 - Calculating MSE")
    from cuml.metrics import mean_squared_error
    score = mean_squared_error(_y_pred, _y_test)
    print("Workflow Complete - RMSE: ", np.sqrt(score))

In [ ]:
client.close()
cluster.close()